In [1]:
from conf import *
from flask import Flask
from flask import request
import tensorflow as tf
import json
import numpy as np
import os
import time
from threading import Thread
from py2neo import Node,Relationship,size,order,Graph,NodeSelector

In [13]:
class KnowGraph(object):
    def __init__(self):
        self.graph = Graph('http://localhost:7474',user='neo4j',password='metadata0')
        self.write_to_statistics()
    #単一のエンティティを見つける
    def lookup_entry(self,client_params,server_param):
        #ネットワーク検索の深さの設定をサポート
        start_time = time.time()
        params=client_params["params"]
        edges=set()
        self.lookup_entry_deep(edges,params,0)
        if len(edges)==0:
            server_param['result']={"success":'false'}
        else:
            server_param['result']={'edges':[list(i) for i in edges],"success":'true'}
            print('今回の検索トリプルの数は:{},時間:{}s'.format(len(edges),time.time()-start_time))

    #2つのエンティティ間の関係を見つける
    def lookup_entry2entry(self,entry1,entry2):

        #　順序を考慮して、4回確認しました
        result1=self.graph.data("MATCH (s)-[r]->(e) where s.name='{}' and e.name='{}' RETURN s.name,r.name,e.name".format(entry1,entry2))
        result2=self.graph.data("MATCH (s)-[r]->(e) where s.name='{}' and e.name='{}' RETURN s.name,r.name,e.name".format(entry2,entry1))

        if len(result1)==0 and len(result2)==0:
            return '関係なし'
        for item in result1:
            print(result1)
            print(result1[0]['r.name'])
        for item in result2:
            print(result2)
            print(result2[0]['r.name'])

    #指定されたエンティティの指定された属性を検索します
    def lookup_entry2property(self,entry,property1):
        result1=self.graph.data("MATCH (s)-[r]->(e) where s.name='{}' and r.name='{}' RETURN s.name,r.name,e.name".format(entry,property1))
        result2=self.graph.data("MATCH (e)<-[r]-(s) where e.name='{}' and r.name='{}' RETURN s.name,r.name,e.name".format(entry,property1))
        if len(result1)==0 and len(result2)==0:
            return '関係なし'
        for item in result1:
            return '1→2に関係あり'
        for item in result2:
            return '1→2に関係あり'

    #クエリ統計
    def lookup_statistics(self,client_params,server_param):
        result=self.graph.data("MATCH (n) RETURN n")
        with open('../data/statistics.txt','r',encoding='utf-8') as f:
            api_nums=f.readline().strip()
        server_param['result']={'total_nums':len(result),'api_nums':api_nums,"success":'true'}

    #APIアクセス時間をカウントする
    def write_to_statistics(self):
        with open('../data/statistics.txt','r',encoding='utf-8') as f:
            api_nums=int(f.readline().strip())+1
        with open('../data/statistics.txt','w',encoding='utf-8') as f:
            f.write(str(api_nums)+'\n')

    #限られた検索の深さ
    def lookup_entry_deep(self,name):
        #現在のルックアップの深さは、必要な深さと等しくてはなりません
        
        result1=self.graph.data("match (s)-[r]->(e) where s.name='{}' return s.name,r.name,e.name".format(name))
        result2=self.graph.data("match (e)<-[r]-(s) where e.name='{}' return s.name,r.name,e.name".format(name))
        if len(result1)==0 and len(result2)==0:
            return
        else:
            return result1, result2

In [14]:
kg=KnowGraph()

In [15]:
kg.lookup_entry2entry('規格','省令')

[{'s.name': '省令', 'r.name': '定める', 'e.name': '規格'}]
定める


In [16]:
kg.lookup_entry2entry('薬局開設者','添付書類')

[{'s.name': '薬局開設者', 'r.name': '提出', 'e.name': '添付書類'}]
提出


In [17]:
kg.lookup_entry2entry('薬局開設者','添付書類')

[{'s.name': '薬局開設者', 'r.name': '提出', 'e.name': '添付書類'}]
提出


In [19]:
kg.lookup_entry_deep('省令')

([{'s.name': '省令', 'r.name': '定める', 'e.name': '区分'},
  {'s.name': '省令', 'r.name': '定める', 'e.name': 'もの'},
  {'s.name': '省令', 'r.name': '定める', 'e.name': '事項'},
  {'s.name': '省令', 'r.name': '定める', 'e.name': '基準'},
  {'s.name': '省令', 'r.name': '定める', 'e.name': '位置'},
  {'s.name': '省令', 'r.name': '定める', 'e.name': '規格'}],
 [])